# تنظیم دقیق مدل‌های Open AI

این دفترچه بر اساس راهنمای فعلی ارائه شده در مستندات [تنظیم دقیق](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) از Open AI تهیه شده است.

تنظیم دقیق عملکرد مدل‌های پایه را برای برنامه شما با آموزش مجدد آن با داده‌ها و زمینه‌های اضافی مرتبط با آن مورد استفاده خاص یا سناریو بهبود می‌بخشد. توجه داشته باشید که تکنیک‌های مهندسی پرامپت مانند _یادگیری چند نمونه‌ای_ و _تولید تقویت‌شده با بازیابی_ به شما امکان می‌دهند پرامپت پیش‌فرض را با داده‌های مرتبط برای بهبود کیفیت تقویت کنید. با این حال، این رویکردها محدود به اندازه پنجره توکن حداکثر مدل پایه هدف هستند.

با تنظیم دقیق، ما در واقع مدل را خود را با داده‌های مورد نیاز دوباره آموزش می‌دهیم (که به ما اجازه می‌دهد تعداد بسیار بیشتری نمونه نسبت به آنچه در پنجره توکن حداکثر جا می‌شود استفاده کنیم) - و نسخه _سفارشی_ از مدل را مستقر می‌کنیم که دیگر نیازی به ارائه نمونه‌ها در زمان استنتاج ندارد. این نه تنها اثربخشی طراحی پرامپت ما را بهبود می‌بخشد (ما انعطاف‌پذیری بیشتری در استفاده از پنجره توکن برای موارد دیگر داریم) بلکه احتمالاً هزینه‌های ما را نیز کاهش می‌دهد (با کاهش تعداد توکن‌هایی که باید در زمان استنتاج به مدل ارسال کنیم).

تنظیم دقیق چهار مرحله دارد:
1. آماده‌سازی داده‌های آموزشی و بارگذاری آن.
1. اجرای کار آموزش برای دریافت مدل تنظیم دقیق شده.
1. ارزیابی مدل تنظیم دقیق شده و تکرار برای کیفیت.
1. استقرار مدل تنظیم دقیق شده برای استنتاج در صورت رضایت.

توجه داشته باشید که همه مدل‌های پایه از تنظیم دقیق پشتیبانی نمی‌کنند - برای آخرین اطلاعات [مستندات OpenAI را بررسی کنید](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst). همچنین می‌توانید یک مدل قبلاً تنظیم دقیق شده را دوباره تنظیم دقیق کنید. در این آموزش، ما از `gpt-35-turbo` به عنوان مدل پایه هدف برای تنظیم دقیق استفاده خواهیم کرد.

---


### مرحله ۱.۱: آماده‌سازی مجموعه داده خود

بیایید یک چت‌بات بسازیم که به شما کمک کند جدول تناوبی عناصر را با پاسخ دادن به سوالات درباره یک عنصر به صورت لیمریک درک کنید. در این آموزش ساده، فقط یک مجموعه داده برای آموزش مدل با چند نمونه پاسخ که قالب مورد انتظار داده‌ها را نشان می‌دهد، ایجاد خواهیم کرد. در یک کاربرد واقعی، شما نیاز دارید مجموعه داده‌ای با نمونه‌های بسیار بیشتری ایجاد کنید. همچنین ممکن است بتوانید از یک مجموعه داده باز (برای حوزه کاربرد خود) در صورت وجود استفاده کرده و آن را برای استفاده در آموزش دقیق بازفرمت کنید.

از آنجا که ما بر روی `gpt-35-turbo` تمرکز داریم و به دنبال پاسخ تک‌مرحله‌ای (تکمیل چت) هستیم، می‌توانیم نمونه‌ها را با استفاده از [این قالب پیشنهادی](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) که الزامات تکمیل چت OpenAI را منعکس می‌کند، ایجاد کنیم. اگر انتظار محتوای مکالمه چندمرحله‌ای دارید، باید از [قالب نمونه چندمرحله‌ای](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) استفاده کنید که شامل پارامتر `weight` برای مشخص کردن اینکه کدام پیام‌ها باید در فرآیند آموزش دقیق استفاده شوند (یا نشوند) است.

ما در اینجا از قالب ساده‌تر تک‌مرحله‌ای برای آموزش خود استفاده خواهیم کرد. داده‌ها در قالب [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) با یک رکورد در هر خط هستند که هر کدام به صورت یک شیء فرمت‌شده JSON نمایش داده شده‌اند. قطعه کد زیر ۲ رکورد را به عنوان نمونه نشان می‌دهد - برای مجموعه نمونه کامل (۱۰ مثال) که برای آموزش دقیق خود استفاده می‌کنیم، به [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) مراجعه کنید. **توجه:** هر رکورد _باید_ در یک خط تعریف شود (نه تقسیم شده در چند خط همانطور که معمولاً در فایل JSON قالب‌بندی شده است)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

در یک کاربرد واقعی، برای نتایج خوب به مجموعه نمونه‌های بسیار بزرگ‌تری نیاز خواهید داشت - تعادل بین کیفیت پاسخ‌ها و زمان/هزینه‌های آموزش دقیق خواهد بود. ما از مجموعه کوچکی استفاده می‌کنیم تا بتوانیم آموزش دقیق را سریع‌تر انجام دهیم و روند را نشان دهیم. برای آموزش دقیق پیچیده‌تر به [این مثال OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) مراجعه کنید.


---

### مرحله ۱.۲ بارگذاری مجموعه داده شما

داده‌ها را با استفاده از API فایل‌ها [همانطور که در اینجا توضیح داده شده است](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) بارگذاری کنید. توجه داشته باشید که برای اجرای این کد، ابتدا باید مراحل زیر را انجام داده باشید:
 - بسته پایتون `openai` را نصب کرده باشید (مطمئن شوید از نسخه‌ای برابر یا بالاتر از ۰.۲۸.۰ برای ویژگی‌های جدید استفاده می‌کنید)
 - متغیر محیطی `OPENAI_API_KEY` را به کلید API اوپن‌ای‌آی خود تنظیم کرده باشید
برای اطلاعات بیشتر، راهنمای [راه‌اندازی](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) ارائه شده برای دوره را ببینید.

اکنون، کد را اجرا کنید تا فایلی برای بارگذاری از فایل JSONL محلی خود ایجاد کنید.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### مرحله ۲.۱: ایجاد کار تنظیم دقیق با استفاده از SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### مرحله ۲.۲: بررسی وضعیت کار

در اینجا چند کاری که می‌توانید با API `client.fine_tuning.jobs` انجام دهید آورده شده است:
- `client.fine_tuning.jobs.list(limit=<n>)` - فهرست آخرین n کار تنظیم دقیق
- `client.fine_tuning.jobs.retrieve(<job_id>)` - دریافت جزئیات یک کار تنظیم دقیق خاص
- `client.fine_tuning.jobs.cancel(<job_id>)` - لغو یک کار تنظیم دقیق
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - فهرست حداکثر n رویداد از کار
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

اولین مرحله از فرآیند _اعتبارسنجی فایل آموزش_ است تا مطمئن شویم داده‌ها در قالب صحیح هستند.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### گام ۲.۳: رویدادها را برای رصد پیشرفت دنبال کنید


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### مرحله ۲.۴: مشاهده وضعیت در داشبورد OpenAI


شما همچنین می‌توانید وضعیت را با بازدید از وب‌سایت OpenAI و بررسی بخش _تنظیم دقیق_ پلتفرم مشاهده کنید. این بخش وضعیت کار جاری را به شما نشان می‌دهد و همچنین به شما امکان می‌دهد تاریخچه اجرای کارهای قبلی را پیگیری کنید. در این تصویر، می‌بینید که اجرای قبلی ناموفق بوده و اجرای دوم موفق شده است. برای زمینه، این اتفاق زمانی رخ داد که اجرای اول از یک فایل JSON با رکوردهای نادرست قالب‌بندی شده استفاده کرد - پس از اصلاح، اجرای دوم با موفقیت کامل شد و مدل را برای استفاده در دسترس قرار داد.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.fa.png)


شما همچنین می‌توانید پیام‌های وضعیت و معیارها را با اسکرول کردن به پایین‌تر در داشبورد بصری مشاهده کنید، همانطور که نشان داده شده است:

| پیام‌ها | معیارها |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.fa.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.fa.png)|


---

### مرحله ۳.۱: بازیابی شناسه و آزمایش مدل تنظیم‌شده در کد


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### مرحله ۳.۲: بارگذاری و آزمایش مدل تنظیم‌شده در محیط Playground

اکنون می‌توانید مدل تنظیم‌شده را به دو روش آزمایش کنید. اول، می‌توانید به محیط Playground مراجعه کرده و از منوی کشویی Models مدل تنظیم‌شده جدید خود را از بین گزینه‌های موجود انتخاب کنید. گزینه دیگر استفاده از گزینه "Playground" است که در پنل Fine-tuning نمایش داده شده است (تصویر بالا را ببینید) که نمای _مقایسه‌ای_ زیر را باز می‌کند و نسخه‌های مدل پایه و مدل تنظیم‌شده را کنار هم برای ارزیابی سریع نشان می‌دهد.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.fa.png)

به سادگی متن زمینه سیستم استفاده شده در داده‌های آموزشی خود را وارد کرده و سوال آزمایشی خود را ارائه دهید. مشاهده خواهید کرد که هر دو طرف با همان زمینه و سوال به‌روزرسانی می‌شوند. مقایسه را اجرا کنید و تفاوت خروجی‌ها را بین آن‌ها خواهید دید. _توجه کنید که مدل تنظیم‌شده پاسخ را در قالبی که در مثال‌های خود ارائه داده‌اید نمایش می‌دهد در حالی که مدل پایه صرفاً از دستور سیستم پیروی می‌کند_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.fa.png)

متوجه خواهید شد که مقایسه همچنین تعداد توکن‌ها برای هر مدل و زمان صرف شده برای استنتاج را نشان می‌دهد. **این مثال خاص یک نمونه ساده است که برای نشان دادن روند است و واقعاً بازتاب‌دهنده یک مجموعه داده یا سناریوی دنیای واقعی نیست**. ممکن است متوجه شوید که هر دو نمونه تعداد توکن‌های یکسانی را نشان می‌دهند (زمینه سیستم و درخواست کاربر یکسان است) در حالی که مدل تنظیم‌شده زمان بیشتری برای استنتاج صرف می‌کند (مدل سفارشی).

در سناریوهای دنیای واقعی، شما از چنین نمونه ساده‌ای استفاده نخواهید کرد، بلکه مدل را بر اساس داده‌های واقعی (مثلاً کاتالوگ محصولات برای خدمات مشتری) تنظیم می‌کنید که کیفیت پاسخ بسیار واضح‌تر خواهد بود. در _آن_ زمینه، دریافت کیفیت پاسخ معادل با مدل پایه نیازمند مهندسی درخواست سفارشی بیشتری خواهد بود که استفاده از توکن‌ها و احتمالاً زمان پردازش مرتبط با استنتاج را افزایش می‌دهد. _برای امتحان این موضوع، نمونه‌های تنظیم مدل در OpenAI Cookbook را بررسی کنید تا شروع کنید._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما در تلاش برای دقت هستیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است حاوی خطاها یا نواقصی باشند. سند اصلی به زبان بومی خود باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حیاتی، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما مسئول هیچ گونه سوءتفاهم یا تفسیر نادرستی که از استفاده این ترجمه ناشی شود، نیستیم.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
